# Database | Mini Project 2

### Imports

In [1]:
# utillities
import pandas as pd
import json
from timeit import timeit
from pprint import pprint

# database drivers
from neo4j import GraphDatabase
from pymongo import MongoClient

## Neo4j

### Get name of persons who acted in a movie in 2006

In [2]:
def neo4j_query_1():
    with driver.session() as session:
        m ='''MATCH(y:Year)-[r:RELEASED]-(m:Movie)-[a:ACTED]-(p:Person)
        WHERE y.year = "2006"
        return p.name'''
        r = session.run(m)

### Get amount of persons that acted in a movie directed by David Yates

In [3]:
def neo4j_query_2():
    with driver.session() as session:
        m ='''MATCH (d:Person)-[:DIRECTED]-(:Movie)-[:ACTED]-(a:Person) 
        WHERE d.name = "David Yates"
        RETURN count(distinct a)'''
        r = session.run(m)

### Get genres Christian Bale appeared in

In [4]:
def neo4j_query_3():
    with driver.session() as session:
        m ='''MATCH (:Person {name:"Christian Bale"})-[:ACTED]-(:Movie)-[:GENRE]-(g:Genre) 
        RETURN count(g), g.name'''
        r = session.run(m)

### Performance Time

In [5]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "1234"))

print("neo4j query 1:", timeit(neo4j_query_1, number=5000))
print("neo4j query 2:", timeit(neo4j_query_2, number=5000))
print("neo4j query 3:", timeit(neo4j_query_3, number=5000))

driver.close()

neo4j query 1: 19.043925968000053
neo4j query 2: 14.960240066999972
neo4j query 3: 13.12418252200007


## Mongo DB

### Get name of persons who acted in a movie in 2006

In [6]:
def mongo_query_1():
    response = mongo_col.aggregate([
        { '$match': { 'Year': 2006 } },
        { '$unwind': '$Actors' },
        { '$group': { '_id': None, 'res': { '$addToSet': '$Actors' } } }
    ])

    return response.next()['res']

### Get amount of persons that acted in a movie directed by David Yates

In [7]:
def mongo_query_2():
    response = mongo_col.aggregate([
        { '$match': { 'Director': 'David Yates' } },
        { '$unwind': '$Actors' },
        { '$group': { '_id': None, 'res': { '$addToSet': '$Actors' } } },
        { '$project': { 'res': { '$size': '$res' } } }
    ]);

    return response.next()['res']

### Get genres Christian Bale appeared in

In [8]:
def mongo_query_3():
    response = mongo_col.aggregate([
        { '$match': { 'Actors': { '$elemMatch': { '$eq': 'Christian Bale' } } } },
        { '$unwind': '$Genre' },
        { '$group': { '_id': None, 'res': { '$addToSet': '$Genre' } } }
    ]);

    return response.next()['res']

### Performance Time

In [9]:
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["mini_project"]
mongo_col = mongo_db["main"]

print("mongo query 1:", timeit(mongo_query_1, number=5000))
print("mongo query 2:", timeit(mongo_query_2, number=5000))
print("mongo query 3:", timeit(mongo_query_3, number=5000))

mongo query 1: 12.316579582000031
mongo query 2: 10.831449104000058
mongo query 3: 11.603583000000071
